In [146]:
from __future__ import print_function
import matplotlib,sys
import math
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [147]:
# Generate Randomised Data dataframe
# 100 records
# Data format -> x0, x1, x2, x3, bias, target(y)

def generate_data():
    data = pd.DataFrame(columns=['x0','x1','x2','x3','target'])
    for i in range(1000):
        x0 = np.random.randint(-1,1)
        x1 = np.random.randint(-1,1)
        x2 = np.random.randint(-1,1)
        x3 = np.random.randint(-1,1)
        # The function which the perceptron is trying to mimic:
        target = 2*x0 + 3*x1 + 4*x2 + x3
        if target > 0:
            target = 1
        else:
            target = 0
        data.loc[i] = [x0,x1,x2,x3,target]
    return data

In [148]:
# Split dataset into training and testing dataset(Dataframe) randomly
def split_data(data):
    train_data = data.sample(frac=0.5,random_state=200)
    test_data = data.drop(train_data.index)
    return train_data,test_data

In [149]:
def predict(inputs,weights, bias, threshold=10):
	activation=0.0
	for i,w in zip(inputs,weights):
		activation += i*w + bias
	return 1 if activation>=threshold else 0

In [150]:
# train a perceptron on generated dataset for n epochs
def train(train_data, epochs, bias, weights, learning_rate, should_plot, verbose=False):
	
    for epoch in range(epochs):
        for index, row in train_data.iterrows():
            inputs = [row['x0'],row['x1'],row['x2'],row['x3']]
            target = row['target']
            prediction = predict(inputs, weights, bias)
            error = target - prediction
            for i in range(len(weights)):
                weights[i] += learning_rate * target * inputs[i]
            bias += learning_rate * target
            if verbose:
                print("Epoch:",epoch,"Error:",error,"Prediction:",prediction,"Weights:",weights)
    
    return [weights, bias]

In [151]:
def test(test_data, weights, bias):
    correct = 0
    for index, row in test_data.iterrows():
        inputs = [row['x0'],row['x1'],row['x2'],row['x3']]
        target = row['target']
        prediction = predict(inputs, weights, bias)
        if prediction == target:
            correct += 1
    return correct/len(test_data)

In [152]:
# plot confusion matrix
def plot_confusion_matrix(true_positive, true_negative, false_positive, false_negative):
    confusion_matrix = np.array([[true_positive, false_negative], [false_positive, true_negative]])
    plt.figure(figsize=(10,10))
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ["Predicted Positive", "Predicted Negative"], rotation=45)
    plt.yticks(tick_marks, ["Actual Positive", "Actual Negative"], rotation=45)
    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

In [153]:
# find accuracy, precision, recall and plot confusion matrix
def performance_metrics(test_data, weights, bias, should_plot=True):
    correct = 0
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for index, row in test_data.iterrows():
        inputs = [row['x0'],row['x1'],row['x2'],row['x3']]
        target = row['target']
        prediction = predict(inputs, weights, bias)
        print("Predicted",prediction,"Actual",target)
        if prediction == target:
            correct += 1
        if prediction == 1 and target == 1:
            true_positive += 1
        if prediction == 0 and target == 0:
            true_negative += 1
        if prediction == 1 and target == 0:
            false_positive += 1
        if prediction == 0 and target == 1:
            false_negative += 1
    accuracy = correct/len(test_data)
    precision = true_positive/(true_positive+false_positive)
    recall = true_positive/(true_positive+false_negative)
    print("True Positive:",true_positive,"True Negative:",true_negative,"False Positive:",false_positive,"False Negative:",false_negative)
    print("Accuracy:",accuracy,"Precision:",precision,"Recall:",recall)
    if should_plot:
        plot_confusion_matrix(true_positive, true_negative, false_positive, false_negative)



In [ ]:
data = generate_data()
train_data, test_data = split_data(data)

#train data
learning_rate = 0.1
epochs = 100
bias = 0
weights = [0,0,0,0]
weights, bias = train(train_data, epochs, bias, weights, learning_rate, True)
print(len(test_data))
print(test(test_data, weights, bias))
performance_metrics(test_data, weights, bias)
